In [2]:
import requests
import os
import csv
import pandas as pd

In [ ]:
# LICENCE
# travail pour QualiFHIR 
# Comité d'éthique : # Comité d'éthique : G2-2024-E012

# 1. tests sur le nouveau serveur de test de Frédérik pour envoyer des ressources FHIR
# 2. développement Python pour travailler avec le modèle de MISTRAL AI : extraction des concepts dans le textre libre 
# 3. validation des codes snomed ct des concepts extraits avec le serveur du Ministère 

# Code Python - Copyright (c) 2025-2026 Dr. Marie DETRAIT at Grand Hôpital de Charleroi

# This script is not intended for sale. 
# Additional Requirements:
# You may not modify the work.
# Any derivative work of this script must include a citation of the original author in any version distributed or 
# publicly displayed of the derivative work.
# Any new work based on this script must be discussed with the author.

In [4]:
# je reprends l'URL du serveur FHIR avec un $lookup comme préconisé par Frédérik 

Server_url = "http://10.30.3.205:8180/fhir/CodeSystem/$lookup"

# bureau et répertoire pour écriture du fichier csv  
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
dico_path = os.path.join(desktop_path, "mes_dicos_valides")
os.makedirs(dico_path, exist_ok=True)  # Crée le répertoire s'il n'existe pas
output_file = os.path.join(dico_path, "dicos_tour_GGA.csv")

# valider code avec le serveur 
def valider_code_snomed(code):
    params = {
        "system": "http://snomed.info/sct",
        "code": code
    }
    response = requests.get(Server_url, params=params) # je fais un get sur le serveur 

    if response.status_code == 200:
        data = response.json()
        # Trouver le paramètre qui s'appelle 'display' principal voir la réponse brute pour le retrouver 
        display_value = None
        active_value = False
        for param in data.get("parameter", []):
            if param.get("name") == "display":
                display_value = param.get("valueString")
            elif param.get("name") == "active":
                active_value = param.get("valueBoolean", False)

        if display_value and active_value:
            return {"valide": True, "display": display_value, "code": code} # ce que je veux voir 
        else:
            return {"valide": False, "raison": "Code inactif ou non trouvé", "code": code} # je gère les exceptions si code non trouvé ou désactivé  
    else:
        return {"valide": False, "raison": f"Erreur {response.status_code}", "code": code} # si erreur du serveur pour avoir l'indication de l'erreur 


# ici la liste en appelant la 1ère fonction 
def valider_liste_codes(liste_codes):
    resultats = []
    for code in liste_codes:
        resultat = valider_code_snomed(code)
        resultats.append(resultat)
    return resultats

# j'enregistre sur mon bureau voir plus haut 
def enregistrer_resultats(resultats, fichier=output_file):
    with open(fichier, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Code SNOMED", "Display", "Statut", "Raison"])
        for r in resultats:
            writer.writerow([
                r.get("code"),
                r.get("display", ""),
                "Valide" if r.get("valide") else "Invalide",
                r.get("raison", "")
            ])


In [7]:
# Exemple d'utilisation avec qqs codes d'hémato
codes = ["52967002", "92818009", "302215000"]
resultats = valider_liste_codes(codes)

In [8]:
# verif 
resultats

[{'valide': True, 'display': 'Myelofibrosis', 'code': '52967002'},
 {'valide': True, 'display': 'Chronic myeloid leukemia', 'code': '92818009'},
 {'valide': True, 'display': 'Thrombocytopenic disorder', 'code': '302215000'}]

In [9]:
enregistrer_resultats(resultats)

print(f"Le fichier CSV a été enregistré ici : {output_file}")

Le fichier CSV a été enregistré ici : C:\Users\dema58815\Desktop\mes_dicos_valides\dicos_tour_GGA.csv


In [12]:
# VERIF 
# Exemple d'utilisation avec qqs codes d'hémato
codes = ["283045001", "92818009", "302215000"]
resultats = valider_liste_codes(codes)

In [13]:
# VERIF -- je ne passe pas "enregistrer_resultats(resultats)"

In [14]:
print(resultats)

[{'valide': False, 'raison': 'Erreur 404', 'code': '283045001'}, {'valide': True, 'display': 'Chronic myeloid leukemia', 'code': '92818009'}, {'valide': True, 'display': 'Thrombocytopenic disorder', 'code': '302215000'}]


In [ ]:
# Xcare : extraction du fichier pour vérification ---> je le mets sur le bureau et je vais le lire ensuite et en faire un DF pandas 

In [ ]:
# Chemin du fichier 
fichier = r"C:\Users\dema58815\Desktop\REMPLACER PAR LE NOM DU FICHIER.csv"

# Lecture du fichier csv et tranformation en DF
df = pd.read_csv(fichier, sep=';', encoding='utf-8')

# Extraction de la colonne "SNOMED CT" avec gestion d' une erreur potentielle 
if "SNOMED CT" in df.columns:
    codes = df["SNOMED CT"].tolist()
    print("\nListe des codes SNOMED CT :")
    print(codes)
else:
    print("\nLa colonne 'SNOMED CT' n'existe pas. Voici les colonnes disponibles :")
    print(df.columns.tolist())

In [37]:
# exemple en pratique : je cherche à valider mes codes des méthodes de diagnostic pour mon DICO Xcare 

codes = ["404684003", "168537006", "16310003", "77477000", "113091000", "82918005", "12894003", "714797009", "25656009", "5880005", "165346000", "250724005", "396785008", "116147009", "396550006", "86273004","234326005", "49401003"]

resultats = valider_liste_codes(codes)
enregistrer_resultats(resultats)

print(f"Le fichier CSV a été enregistré ici : {output_file}")

Le fichier CSV a été enregistré ici : C:\Users\dema58815\Desktop\mes_dicos_valides\dicos_methode.csv


In [38]:
resultats

[{'valide': True, 'display': 'Clinical finding', 'code': '404684003'},
 {'valide': True, 'display': 'Plain X-ray', 'code': '168537006'},
 {'valide': True, 'display': 'Ultrasonography', 'code': '16310003'},
 {'valide': True, 'display': 'Computed tomography', 'code': '77477000'},
 {'valide': True,
  'display': 'Magnetic resonance imaging',
  'code': '113091000'},
 {'valide': True,
  'display': 'Positron emission tomography',
  'code': '82918005'},
 {'valide': True, 'display': 'Functional assessment', 'code': '12894003'},
 {'valide': True, 'display': 'Histologic test', 'code': '714797009'},
 {'valide': True,
  'display': 'Physical examination, complete',
  'code': '25656009'},
 {'valide': True, 'display': 'Physical examination', 'code': '5880005'},
 {'valide': True,
  'display': 'Laboratory test result abnormal',
  'code': '165346000'},
 {'valide': True, 'display': 'Tumor marker measurement', 'code': '250724005'},
 {'valide': True,
  'display': 'Histologic type of metastatic neoplasm',
  

In [ ]:
# TEST pour Mouscron

Test pour Mouscron 

Bonjour dr Detrait,

Je me demandais s'il serait possible d'ajouter dans les motifs de demande d'avis infectio, un encart "tour GGA" .

Ce motif correspondrait à l'intervention de l'infectiologue suite à la mise en évidence au laboratoire ou en pharmacie d'un problème tel que :
- communication d'une hémoculture positive ou d'un autre résultat microbiologique
- posologie antibiotique inadéquate ou choix de molécule antibiotique non adéquat

Merci,
Clotilde

Bonjour Clotilde, 

voici ce que j'ai trouvé dans la Snomed Ct  = les codes à valider dans ce script 

qui peut répondre : 

//395067002//  Optimization of drug dosage, optimisation d'une posologie 
//410299006// éducation, orientation et conseil sur les résultats de laboratoire ---> pour mise en évidence au laboratoire d'un problème tel que : communication d'une hémoculture positive, d'un autre résultat microbiologique, etc.
//473232002//  Révision d'un choix de médicament pour une maladie infectieuse

j'ai ajouté aussi : "TOUR GGA" qui envoie vers Specialist multidisciplinary team (qualifier value)
//408458006// 

j'ai tout ajouté dans le DICO motif avis en infectio


In [39]:

codes = ["395067002", "410299006", "473232002", "408458006"]
resultats = valider_liste_codes(codes)
enregistrer_resultats(resultats)

# print(f"Le fichier CSV a été enregistré ici : {output_file}")

In [40]:
resultats

[{'valide': True,
  'display': 'Optimization of drug dosage',
  'code': '395067002'},
 {'valide': True,
  'display': 'Lab findings education, guidance, and counseling',
  'code': '410299006'},
 {'valide': True,
  'display': 'Infectious disease medication review',
  'code': '473232002'},
 {'valide': True,
  'display': 'Specialist multidisciplinary team',
  'code': '408458006'}]

In [ ]:
# dico tour GGA 

In [15]:
url = "http://10.30.3.205:8180/fhir/ValueSet/$expand"

# requête pour développer la thrombopénie et tous ses descendants
payload = {
  "resourceType": "Parameters",
  "parameter": [
    {
      "name": "valueSet",
      "resource": {
        "resourceType": "ValueSet",
        "status": "active",
        "compose": {
          "include": [
            {
              "system": "http://snomed.info/sct",
              "filter": [
                {
                  "property": "concept",
                  "op": "is-a",
                  "value": "302215000"  # Code SNOMED CT pour la thrombopénie
                }
              ]
            }
          ]
        }
      }
    },
    {
      "name": "displayLanguage",
      "valueString": "en"  # Pour les libellés en anglais (attention Frédérik m'a dit uniquement en anglais)
    },
    {
      "name": "includeDesignations",
      "valueBoolean": True  # Pour inclure les synonymes
    }
  ]
}

# En-têtes HTTP
headers = {
    "Content-Type": "application/fhir+json",
    "Accept": "application/fhir+json"
}

# Envoi de la requête
response = requests.post(url, json=payload, headers=headers)

# Traitement de la réponse
if response.status_code == 200:
    expanded_valueset = response.json()
    for concept in expanded_valueset["expansion"]["contains"]:
        print(f"Code: {concept['code']}, Libellé: {concept['display']}")
else:
    print(f"Erreur: {response.status_code}, {response.text}")

Code: 16740451000119109, Libellé: Postpartum hemolysis-elevated liver enzymes-low platelet count syndrome
Code: 16623961000119100, Libellé: Pancytopenia caused by immunosuppressant
Code: 10752381000119101, Libellé: Foetal thrombocytopenia
Code: 328391000119108, Libellé: Pancytopenia induced by antidiabetics
Code: 328381000119105, Libellé: Pancytopenia caused by anticonvulsant
Code: 328371000119107, Libellé: Pancytopenia caused by antithyroid drug
Code: 328331000119109, Libellé: Pancytopenia caused by nonsteroidal anti-inflammatory agent
Code: 328321000119106, Libellé: Pancytopenia caused by colchicine
Code: 328301000119102, Libellé: Pancytopenia due to antineoplastic chemotherapy
Code: 118791000119106, Libellé: Aplastic anaemia caused by antineoplastic agent
Code: 97571000119109, Libellé: Thrombocytopenia co-occurrent and due to alcoholism
Code: 1360083000, Libellé: A rare unclassified autoinflammatory syndrome characterised by neonatal onset pancytopenia, type I interferon-dependent m

In [16]:
import pandas as pd

url = "http://10.30.3.205:8180/fhir/ValueSet/$expand"

payload = {
  "resourceType": "Parameters",
  "parameter": [
    {
      "name": "valueSet",
      "resource": {
        "resourceType": "ValueSet",
        "status": "active",
        "compose": {
          "include": [
            {
              "system": "http://snomed.info/sct",
              "filter": [
                {
                  "property": "concept",
                  "op": "is-a",
                  "value": "302215000"  # Thrombopénie
                }
              ]
            }
          ]
        }
      }
    },
    {
      "name": "displayLanguage",
      "valueString": "en"
    },
    {
      "name": "includeDesignations",
      "valueBoolean": True
    }
  ]
}

headers = {
    "Content-Type": "application/fhir+json",
    "Accept": "application/fhir+json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    expanded_valueset = response.json()

    # Extraire les concepts dans un DataFrame
    concepts = expanded_valueset["expansion"]["contains"]
    df = pd.DataFrame(concepts)

    # Sélectionner uniquement les colonnes utiles (par exemple, code et display)
    df = df[["code", "display"]]

    # Afficher le DataFrame
    print(df.head())

    # Exporter en CSV sur mon bureau
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    csv_path = os.path.join(desktop_path, "thrombopenie_concepts.csv")
    df.to_csv(csv_path, index=False)

    print(f"Fichier CSV enregistré sur le bureau : {csv_path}")
else:
    print(f"Erreur: {response.status_code}, {response.text}")

                code                                            display
0  16740451000119109  Postpartum hemolysis-elevated liver enzymes-lo...
1  16623961000119100           Pancytopenia caused by immunosuppressant
2  10752381000119101                            Foetal thrombocytopenia
3    328391000119108              Pancytopenia induced by antidiabetics
4    328381000119105              Pancytopenia caused by anticonvulsant
Fichier CSV enregistré sur le bureau : C:\Users\dema58815\Desktop\thrombopenie_concepts.csv


In [ ]:
# FIN 